In [ ]:
import requests
import boto3
import os
import pytz
from datetime import datetime, timezone
from botocore.credentials import RefreshableCredentials
from botocore.session import get_session

# Use API key to obtain a temporary access token (for authentication & authorization with Curation API routes)

api_key = open("../my_key").read().strip()  # SUBSTITUTE VALUE HERE -- path to file containing your API key
access_token_headers = {"x-api-key": api_key}
access_token_url = "https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/auth/token"
resp = requests.post(access_token_url, headers=access_token_headers)
access_token = resp.json().get("access_token")

print(access_token)

In [ ]:
# Use access token to obtain temporary s3 write credentials for a given Collection; these credentials will only work for THIS Collection.

collection_id = "e0b51cd8-0e54-46d2-a163-5e4787f472fe"  # SUBSTITUTE VALUE HERE -- your (non-public) Collection id, e.g. "1234abcd-5678-efab-1a2b-3c4d5e6f8a9b"
s3_credentials_url = f"https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/collections/{collection_id}/datasets/s3-upload-credentials"
s3_cred_headers = {"Authorization": f"Bearer {access_token}"}

time_zone_info = datetime.now(timezone.utc).astimezone().tzinfo

def retrieve_s3_credentials():
    resp = requests.post(s3_credentials_url, headers=s3_cred_headers)
    s3_creds = resp.json().get("Credentials")
#     print(s3_creds.get("Expiration"))
#     print()
    s3_creds_formatted = {
        "access_key": s3_creds.get("AccessKeyId"),
        "secret_key": s3_creds.get("SecretAccessKey"),
        "token": s3_creds.get("SessionToken"),
        "expiry_time": datetime.fromtimestamp(s3_creds.get("Expiration")).replace(tzinfo=time_zone_info).isoformat(),
    }
#     t = datetime.fromtimestamp(s3_creds.get("Expiration"))
#     print(t.isoformat())
#     print(t.tzinfo)
    return s3_creds_formatted

session_creds = RefreshableCredentials.create_from_metadata(
    metadata=retrieve_s3_credentials(),
    refresh_using=retrieve_s3_credentials,
    method="sts-assume-role",
)
print(session_creds)
retrieve_s3_credentials()

In [ ]:
# Upload file using temporary s3 credentials

# access_key_id, secret_access_key, session_token = s3_creds["AccessKeyId"], s3_creds["SecretAccessKey"], s3_creds["SessionToken"]
session = get_session()
session._credentials = session_creds
boto3_session = boto3.Session(botocore_session=session)

s3 = boto3_session.client("s3")
filename = "/Users/danielhegeman/Downloads/c114c20f-1ef4-49a5-9c2e-d965787fb90c_f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad"  # SUBSTITUTE VALUE HERE -- Designate your datafile to upload; fully-qualified path
curator_tag = "large.h5ad"  # SUBSTITUTE VALUE HERE -- **MUST POSSESS '.h5ad' SUFFIX**; will serve as unique identifier for the resultant Dataset *within* this Collection

filesize = os.path.getsize(filename)
import threading
def get_cb():
    lock = threading.Lock()
    total = 0

    def cb(num_bytes):
        nonlocal total
        lock.acquire()
        total += num_bytes
        lock.release()
        print(f"{total / filesize}\r", end="")

    return cb


try:
    success = s3.upload_file(
        Filename=filename,
        Bucket="cellxgene-dataset-submissions-dev",
        Key=f"{collection_id}/{curator_tag}",
        Callback=get_cb(),
    )
except Exception as e:
    print(e)
else:
    print(f"File {filename} successfully uploaded to Collection {collection_id} with tag {curator_tag}")